RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [1]:
import os
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


d:\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  âœ“ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  âœ— Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 4 PDF files to process

Processing: 12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization).pdf
  âœ“ Loaded 19 pages

Processing: RAG_Research_Paper_arkiv.pdf
  âœ“ Loaded 21 pages

Processing: Resume_Latest.pdf
  âœ“ Loaded 1 pages

Processing: __The_Ultimate_AI_Engineering_Learning_Roadmap__Fr[1].pdf
  âœ“ Loaded 11 pages

Total documents loaded: 52


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'WeasyPrint 65.1', 'creator': 'ChatGPT', 'creationdate': '', 'title': '12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization)', 'author': 'ChatGPT Deep Research', 'source': '..\\data\\pdf\\12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization).pdf', 'total_pages': 19, 'page': 0, 'page_label': '1', 'source_file': '12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization).pdf', 'file_type': 'pdf'}, page_content='12-Month Roadmap to Become a Production-Ready\nAI Engineer (Agentic AI Specialization)\nOverview: This roadmap is tailored for Yash – a 4th-year ECE student with basic Python, math, and ML\nknowledge – to transform into a production-ready AI Engineer specialized in Agentic AI over 12 months.\nYash  will  dedicate  ~8  hours  daily.  The  plan  is  divided  into  monthly  phases  with  clear  goals,  hands-on\nprojects,  and  curated  resources.  To  minimize  

In [4]:
### Text Splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split Resumes into smaller chunks for better performance"""

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n","\n"," ",""]

    )

    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content : {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    return split_docs


In [5]:
chunks=split_documents(all_pdf_documents)
chunks


Split 52 documents into 263 chunks

Example chunk:
Content : 12-Month Roadmap to Become a Production-Ready
AI Engineer (Agentic AI Specialization)
Overview: This roadmap is tailored for Yash – a 4th-year ECE student with basic Python, math, and ML
knowledge – t...
Metadata: {'producer': 'WeasyPrint 65.1', 'creator': 'ChatGPT', 'creationdate': '', 'title': '12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization)', 'author': 'ChatGPT Deep Research', 'source': '..\\data\\pdf\\12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization).pdf', 'total_pages': 19, 'page': 0, 'page_label': '1', 'source_file': '12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization).pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'WeasyPrint 65.1', 'creator': 'ChatGPT', 'creationdate': '', 'title': '12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization)', 'author': 'ChatGPT Deep Research', 'source': '..\\data\\pdf\\12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization).pdf', 'total_pages': 19, 'page': 0, 'page_label': '1', 'source_file': '12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization).pdf', 'file_type': 'pdf'}, page_content='12-Month Roadmap to Become a Production-Ready\nAI Engineer (Agentic AI Specialization)\nOverview: This roadmap is tailored for Yash – a 4th-year ECE student with basic Python, math, and ML\nknowledge – to transform into a production-ready AI Engineer specialized in Agentic AI over 12 months.\nYash  will  dedicate  ~8  hours  daily.  The  plan  is  divided  into  monthly  phases  with  clear  goals,  hands-on\nprojects,  and  curated  resources.  To  minimize  

### Embeddings and VectorDB

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict , Any , Tuple
from sklearn.metrics.pairwise import cosine_similarity


In [7]:
from typing import List
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000021B5160FF50>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: e96498a6-92dc-46c8-ba0e-59ed4edb4d75)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000021B51618E90>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 5aab82b4-acba-4a3f-bed7-521ba70459a5)')' thrown while requesting HEAD https://huggingface.

Model loaded successfully. Embedding dimension: 384


### VectorStore


In [8]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self,documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        print(f"Adding {len(documents)}documents to vector store...")

        #Prepare data for ChromaDB
        ids =[]
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc,embedding) in enumerate(zip(documents,embeddings)):
            #Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            #prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['context_length'] = len(doc.page_content)
            metadatas.append(metadata)

            #Document content
            documents_text.append(doc.page_content)

            #Embedding
            embeddings_list.append(embedding.tolist())

        #Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                documents=documents_text
                
            )
            print(f"Successfully added {len(documents)}documents to vector store...")
            print(f"Total document in collection: {self.collection.count()}")
        
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise
vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 718


In [9]:
chunks

[Document(metadata={'producer': 'WeasyPrint 65.1', 'creator': 'ChatGPT', 'creationdate': '', 'title': '12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization)', 'author': 'ChatGPT Deep Research', 'source': '..\\data\\pdf\\12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization).pdf', 'total_pages': 19, 'page': 0, 'page_label': '1', 'source_file': '12-Month Roadmap to Become a Production-Ready AI Engineer (Agentic AI Specialization).pdf', 'file_type': 'pdf'}, page_content='12-Month Roadmap to Become a Production-Ready\nAI Engineer (Agentic AI Specialization)\nOverview: This roadmap is tailored for Yash – a 4th-year ECE student with basic Python, math, and ML\nknowledge – to transform into a production-ready AI Engineer specialized in Agentic AI over 12 months.\nYash  will  dedicate  ~8  hours  daily.  The  plan  is  divided  into  monthly  phases  with  clear  goals,  hands-on\nprojects,  and  curated  resources.  To  minimize  

In [10]:
### convert the text to embeddings
texts=[doc.page_content for doc in chunks]

##Generate the embeddings

embeddings = embedding_manager.generate_embeddings(texts)

##Store in the vector database

vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 263 texts...


Batches: 100%|██████████| 9/9 [00:10<00:00,  1.18s/it]


Generated embeddings with shape: (263, 384)
Adding 263documents to vector store...
Successfully added 263documents to vector store...
Total document in collection: 981


### retriever pipeline from vector store


In [11]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

         # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)


In [12]:
rag_retriever.retrieve("What is New Modules in Modular RAG")

Retrieving documents for query: 'What is New Modules in Modular RAG'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 37.57it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_d2735ae3_110',
  'content': 'a key strategy. This concept has been implemented in frame-\nworks such as LlamaIndex 2, LangChain3, and HayStack [12].\nFeeding all relevant documents directly into LLMs can lead\nto information overload, diluting the focus on key details with\nirrelevant content.To mitigate this, post-retrieval efforts con-\ncentrate on selecting the essential information, emphasizing\ncritical sections, and shortening the context to be processed.\n2https://www.llamaindex.ai\n3https://www.langchain.com/\nC. Modular RAG\nThe modular RAG architecture advances beyond the for-\nmer two RAG paradigms, offering enhanced adaptability and\nversatility. It incorporates diverse strategies for improving its\ncomponents, such as adding a search module for similarity\nsearches and refining the retriever through fine-tuning. Inno-\nvations like restructured RAG modules [13] and rearranged\nRAG pipelines [14] have been introduced to tackle specific\nchallenges. The shift to